# TODO

## Task 2: Retrieval over corpus
Compute pdist2

- Extract features from MCN

    `bash dump_features.sh`
    
- Move features

    `mv ../localizing-moments/results/*.hdf5 data/interim/mcn/features/`

- Class to parse and interact with corpus
    - ~~reading hdf5~~
    - ~~make dictionary~~
    - ~~~make corpus matrix~~
    - ~~Method to return video and segment~~
    - ~~grab possible segments~~
    - optional: reading ground-truth

- Compute distance and retrieve sorted samples for a given vector query
    - optional: batch computation

In [1]:
import itertools
import json
from collections import OrderedDict

import h5py
import numpy as np

POSSIBLE_SEGMENTS = [(0,0), (1,1), (2,2), (3,3), (4,4), (5,5)]
for i in itertools.combinations(range(6), 2):
    POSSIBLE_SEGMENTS.append(i)


class Corpus(object):
    """Corpus of videos with clips of interest to index
    
    TODO
        batch indexing
    """
    
    def __init__(self, filename, segments=POSSIBLE_SEGMENTS):
        self.segments = np.array(segments)
        self._create_repo(filename)
        self._create_feature_matrix()
    
    def ind_to_repo(self, i):
        "retrieve video and segment index"
        # purpose: given index in matrix return video
        video_index = i // self.T
        segment_index = i % self.T
        return video_index, segment_index
    
    def search(self, x):
        distance = ((self.features - x)**2).sum(axis=1)
        distance_sorted_idx = np.argsort(distance)
        distance_sorted = distance[distance_sorted_idx]
        video_idx, segment_idx = self.ind_to_repo(distance_sorted_idx)
        return video_idx, segment_idx, distance_sorted
    
    def _create_repo(self, filename):
        "read hdf5 and make corpus repo"
        self.container = OrderedDict()
        with h5py.File(filename, 'r') as f:
            for k, v in f.items():
                self.container[k] = v[:]
        self.videos = np.array(list(self.container.keys()))
        self.T, self.D = self._grab_sample_value().shape
        self.num_videos = len(self.videos)
        assert self.T == len(self.segments)
    
    def _create_feature_matrix(self):
        "make corpus matrix"
        # purpose: perform search without loop over repo
        dtype = self._grab_sample_value().dtype
        self.features = np.empty((self.num_videos * self.T, self.D),
                               dtype=dtype)
        for i, (_, v) in enumerate(self.container.items()):
            r_start = i * self.T
            r_end = r_start + self.T
            self.features[r_start:r_end, :] = v
    
    def _grab_sample_value(self, idx=0):
        sample_key = self.videos[idx]
        return self.container[sample_key]
    

file_corpus = 'data/interim/mcn/features/corpus_val.hdf5'
val_corpus = Corpus(file_corpus)
# test video and segment mapping
index = np.random.randint(len(val_corpus.features))
print(index, val_corpus.ind_to_repo(index))

file_queries = 'data/interim/mcn/features/queries_val.hdf5'
file_annotations = 'data/raw/val_data.json'

with h5py.File(file_queries, 'r') as fid:
    sample_key = list(fid.keys())[0]
    sample_value = fid[sample_key][:]

val_corpus.search(sample_value)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


3076 (146, 10)


(array([432, 402, 975, ..., 934, 226,   7]),
 array([ 2,  4,  3, ..., 10, 10, 10]),
 array([0.38135153, 0.39323872, 0.40213436, ..., 1.4829735 , 1.550116  ,
        1.5524127 ], dtype=float32))

## Task 3: Implement evaluation code

The metric should reflect the probability of finding a relevant moment, clip inside a video, for a given query `q` among `k` possible moments in the entire corpus.

Given that he have multiple annotations per query inside the same video (accounting for inherent temporal ambiguity), we need a "concensus" or "thresholding" criteria to assess that a given clip is relevant among a pool of annotations.

Note: This is not related to the tIOU threshold. Only a single, golden, annotation would circumvent this problem.

Concensus - thresholding strategies:
- max. Assigns a true if the prediction match any annotation.
Makes problem easier as the chance increases proportionally to ambiguity of query. It's sensitive to outliers in annotation process. Probably relevant for tIOU.

Metrics

- R@k,c

    prob of finding a moment on `top-k`.
    Here, the moment is relevant when its average rank on the best 3 out of 4 annotations is lower or equal than `j`.

- mIOU

   
- mRank
    mean rank.


~~Among the thresholding strategies, we have:~~

~~- average. Makes problem harder as it forces to agree with multiple annotators. It's also sensitive to outliers.~~

~~- average over a subset of annotations. As above but accounting for outliers.~~

~~Thresholding implies to compute a given metric a measure the~~
~~For consistency with DiDeMo standard, we opt for thresholding:~~

Implementation note: given multiple annotations for each query is not conveninent to deal with the raw indexes from the feature matrix. Moreover, those indexes would be useless for considering tIOU. Hopefully, we have a function to invert those indexes into video and segment indexes. However, we need to keep consistency in the list of videos and segments to make an apple 2 apple comparsion.

Evaluation pseudo-code

```
inputs: list of queries; ground-truth, k

recall_at_k = []
miou = []
for each query:
    get vector
    compute distance and return sorted list of indexes
    # prediction would end here
    # code for server could continue to provide more info.
    # like red to miss and green for hit ;)
    
    # evaluation
    for i in range(k)
        if video_idx[i] == gt[query][video_idx]:
            check if it's the segment we are looking for
            miou.append()
        else:
            miou.append(0)

sum(recall_at_k) / len(recall_at_k)
sum(miou_at_k) / len(miou_at_k)
```

TODO: what is the average iou in this case? max/mean - average_iou

DiDeMo evaluation

```python
average_ranks = []
average_iou = []
for s, d in zip(segments, data):
  pred = s[0]
  ious = [iou(pred, t) for t in d['times']]
  average_iou.append(np.mean(np.sort(ious)[-3:]))
  ranks = [rank(s, t) for t in d['times']]
  average_ranks.append(np.mean(np.sort(ranks)[:3]))
rank1 = np.sum(np.array(average_ranks) <= 1)/float(len(average_ranks))
rank5 = np.sum(np.array(average_ranks) <= 5)/float(len(average_ranks))
miou = np.mean(average_iou)
```

a. Task a
- ~~data structure for ground-truth.~~
- todo: consistency for indexing. avoid to 

b. evaluation


In [2]:
class GTQueries():
    "Pool of queries with ground-truth"
    
    def __init__(self, filename, videos, segments):
        self.data = OrderedDict()
        self.filename = filename
        self.diff_four = [0, 0]  # debugging
        self._sanitize(videos, segments)
        self._setup_from_file(filename)
        
    def _add_list_items(self, l):
        for i in l:
            query_id = i.pop('annotation_id')
            i['video_index'] = self.lookup_video[i['video']]
            i['segment_indices'] = self._get_segment_indexes(i['times'])
            self.data[query_id] = i
        self.query_ids = list(self.data.keys())
        
    def _get_segment_indexes(self, annotations):
        idxs = np.array([self.lookup_segment[tuple(i)] for i in annotations])
        # debugging
        if len(annotations) > 4:
            self.diff_four[0] += 1
        elif len(annotations) < 4:
            self.diff_four[1] += 1
        return idxs
        
    def _sanitize(self, videos, segments):
        if videos is None or segments is None:
            raise ValueError
        assert len(set(videos)) == len(videos)
        assert len(set(segments)) == len(segments)
        self.lookup_video = dict(zip(videos, range(len(videos))))
        self.lookup_segment = dict(zip(segments, range(len(segments))))
        
    def _setup_from_file(self, filename):
        with open(filename, 'r') as f:
            self._add_list_items(json.load(f))
    
    def __getitem__(self, idx):
        return self.data[idx]


segments = list(map(tuple, val_corpus.segments.tolist()))
val_gt = GTQueries(file_annotations,
                   val_corpus.videos.tolist(),
                   segments)
val_gt[0]

{'description': 'a man on a sled waves as he goes past the camera.',
 'dl_link': 'https://www.flickr.com/video_download.gne?id=4273219306',
 'num_segments': 6,
 'segment_indices': array([ 3,  3,  2, 15]),
 'times': [[3, 3], [3, 3], [2, 2], [2, 3]],
 'video': '52614599@N00_4273219306_d2d89ed9f0.avi',
 'video_index': 701}

In [3]:
def intersection(segments1, segments2):
    """Compute pairwise intersection length between segments.
    
    Args:
        segments1 (numpy array): shape [N, 2] holding N segments
        segments2 (numpy array): shape [M, 2] holding M segments
    Returns:
        a numpy array with shape [N, M] representing pairwise intersection length
    """
    [t_min1, t_max1] = np.split(segments1, 2, axis=1)
    [t_min2, t_max2] = np.split(segments2, 2, axis=1)

    all_pairs_min_tmax = np.minimum(t_max1, np.transpose(t_max2))
    all_pairs_max_tmin = np.maximum(t_min1, np.transpose(t_min2))
    intersect_length = np.maximum(
        np.zeros(all_pairs_max_tmin.shape),
        all_pairs_min_tmax - all_pairs_max_tmin)
    return intersect_length


def length(segments):
    """Computes length of segments.
    
    Args:
        segments (numpy array): shape [N, 2] holding N segments
    Returns:
        a numpy array with shape [N] representing segment length
    Note:
        it works with time, it would be off if using frames.
    """
    return segments[:, 1] - segments[:, 0]


def iou(segments1, segments2):
    """Computes pairwise intersection-over-union between box collections.

    Args:
        segments1 (numpy array): shape [N, 2] holding N segments
        segments2 (numpy array): shape [M, 4] holding N boxes.
    Returns:
        a numpy array with shape [N, M] representing pairwise iou scores.
    """
    intersect = intersection(segments1, segments2)
    length1 = length(segments1)
    length2 = length(segments2)
    union = np.expand_dims(length1, axis=1) + np.expand_dims(
        length1, axis=0) - intersect
    return intersect / union

In [4]:
class RetrievalEvaluation():
    "TODO: count search for a given query_id"
    
    def __init__(self, corpus_h5, groundtruth_json,
                 k=(1,), iou_threshold=0.75):
        self.corpus = Corpus(corpus_h5)
        videos = self.corpus.videos.tolist()
        segments = list(map(tuple, self.corpus.segments.tolist()))
        self._precompute_iou()
        self.gt_queries = GTQueries(groundtruth_json,
                                    videos=videos, segments=segments)
        self.k = k
        self.iou_threshold = 0.75
        self.reset()
        self._k = np.array(self.k)
        
    def eval(self):
        recall_at_k = [sum(i) / len(i) for i in self.hit_k]
        recall_k_iou = [sum(i) / len(i) for i in self.hit_k_iou]
        miou = sum(self.miou) / len(self.miou)
        mrank = np.mean(self.rank)
        return recall_at_k, recall_k_iou, miou, mrank
        
    def eval_single_query(self, query, query_id):
        # todo encode language vector
        raise NotImplementedError
    
    def eval_single_vector(self, vector, query_id):
        if query_id not in self.gt_queries.data:
            # log-this
            pass
        (pred_video_indices, pred_segment_indices,
         sorted_dist) = self.corpus.search(vector)
        true_video = self.gt_queries[query_id]['video_index']
        true_segments = self.gt_queries[query_id]['segment_indices']
        
        # Q&D -> tp_fp_segments
        # Note: I keep the entire corpus to compute rank
        # Note: np.in1d won't generalize for other criteria
        # TODO-p: check if bottleneck. In theory, using a loop
        #         and break for fp from other videos sounds efficient.
        tp_fp_videos = pred_video_indices == true_video
        tp_fp_segments = np.in1d(pred_segment_indices, true_segments)
        tp_fp_labels = np.logical_and(tp_fp_videos, tp_fp_segments)
        for i, k in enumerate(self.k):
            self.hit_k[i].append(tp_fp_labels[:k].sum(dtype=bool))
        self.rank.append(np.where(tp_fp_labels)[0].min())
        
        # Q&D -> mIOU
        self.miou.append(0)
        if tp_fp_labels[0]:
            ious = self.iou_matrix[true_segments, pred_segment_indices[0]]
            self.miou[-1] = np.mean(np.sort(ious)[-3:])
            
        # Q&D -> R@k,tIOU
        max_k = max(self.k)
        topk_tp_fp_videos = tp_fp_videos[:max_k]
        topk_pred_segment_indices = pred_segment_indices[:max_k]
        i, j = np.meshgrid(true_segments, topk_pred_segment_indices)
        iou = np.max(topk_tp_fp_videos[:, None] * self.iou_matrix[i, j],
                     axis=1)
        topk_tp_fp_labels = iou >= self.iou_threshold
        cum_tp_fp_labels = topk_tp_fp_labels.cumsum()
        for i, k in enumerate(self.k):
            self.hit_k_iou[i].append(cum_tp_fp_labels[k - 1] > 0)
            if self.hit_k[i][-1] != self.hit_k_iou[i][-1]:
                print(i, k)
                print(self.hit_k[i][-1], self.hit_k_iou[i][-1])
                print(tp_fp_labels[:k])
                print(topk_tp_fp_labels[:k])
                print(cum_tp_fp_labels[:k])
                print(cum_tp_fp_labels[k - 1] > 0)
                raise
    
    def reset(self):
        self.hit_k = [[] for i in self.k]
        self.hit_k_iou = [[] for i in self.k]
        self.miou = []
        self.rank = []
        
    def _precompute_iou(self):
        segments = self.corpus.segments * 5
        segments[:, 1] += 5
        self.iou_matrix = iou(segments, segments)
    

val_judge = RetrievalEvaluation(file_corpus, file_annotations, (1, 5, 10))
file_queries = 'data/interim/mcn/features/queries_val.hdf5'

with h5py.File(file_queries, 'r') as fid:
    # sample_key = list(fid.keys())[0]
    # sample_value = fid[sample_key][:]
    # query_id = int(sample_key)
    # val_judge.eval_single_vector(sample_value, query_id)
    import time
    start = time.time()
    for sample_key, h5ds in fid.items():
        query_id = int(sample_key)
        query_vector = h5ds[:]
        val_judge.eval_single_vector(query_vector, query_id)
    performace = val_judge.eval()
    print('r@{0:}={2:};\nr@{0:},{1:}={3:};\nmRank={5:.2f}\nmIOU={4:.4f};'
          .format(val_judge.k, val_judge.iou_threshold,
                  *performace))
    print('Elapsed time:', time.time() - start)    
# to run the evaluation again
# val_judge.reset()

r@(1, 5, 10)=[0.004545454545454545, 0.014114832535885167, 0.027033492822966507];
r@(1, 5, 10),0.75=[0.004545454545454545, 0.014114832535885167, 0.027033492822966507];
mRank=1657.46
mIOU=0.0043;
Elapsed time: 23.846515893936157
